In [1]:
import sys
import os
import numpy as np
import multiprocessing
import dill
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from util_functions import *
from uncertainty_analysis import *
from sim_helper_functions import *


module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
from load_params import load_params

from multi_group_simulation import MultiGroupSimulation

In [2]:
NPOINTS = 13
MULT_UB = 3
MULT_LB = 1

In [3]:
lhs_output_sim_files = []
for i in range(2000):
    fname = '/home/jmc678/covid_data/group-testing/notebooks/apr_29_scenarios/point_{}.dill'.format(i)
    lhs_output_sim_files.append(fname)

scenario_data = load_sim_output(lhs_output_sim_files)
res_results = residential_regression(scenario_data)
res_pessimistic = calculate_pessimistic_scenario(res_results)
centre = get_centre_point()


PermissionError: [Errno 13] Permission denied: '/home/jmc678/covid_data/group-testing/notebooks/apr_29_scenarios/point_0.dill'

The following function computes the new pessimistic configuration from the hyperplane calculation. It returns $\gamma$ and $x_{pess}$.

$$
\gamma = 1.645 \frac{\sqrt{c^T\Sigma c}\sqrt{c^T\Sigma^3c}}{c^T\Sigma^2c} 
= 1.645\frac{\sqrt{\sum_ic_i^2\sigma_i^2}{\sqrt{\sum_ic_i^2\sigma_i^6}}}{\sum_ic_i^2\sigma_i^4}.
$$

$$
x_{pess} = \mu +  1.645 \frac{\sqrt{c^T\Sigma c}\sqrt{c^T\Sigma^3c}}{c^T\Sigma^2c}\frac{\Sigma c}{\sqrt{c^T\Sigma c}} = \mu +  1.645 \frac{\sqrt{c^T\Sigma^3c}\Sigma c}{c^T\Sigma^2c}.
$$

One can directly compare $\gamma$ with 1.96 to see how differently the two pessimistic configurations proceed in the pessimistic direction.

In [ ]:
def calculate_pessimistic_scenario_hyperplane(results, invquantile = 1.645):
    # the keys in dict(results.params) specify whether this is for residential
    # or virtual vs. residential
    lr_results = dict(results.params)
    range_dict = dict()
    pess_direction = dict()
    params = set(lr_results.keys()) - set(['const'])
    for param in params:
        range_dict[param] = (PARAM_BOUNDS[param][1] - PARAM_BOUNDS[param][0])/2

    
    sum_squares_Sigma_1 = 0
    sum_squares_Sigma_2 = 0
    sum_squares_Sigma_3 = 0
    
    for param in params:
        sum_squares_Sigma_1 += (lr_results[param]*range_dict[param]) ** 2
        sum_squares_Sigma_2 += (lr_results[param]* range_dict[param]**2) ** 2
        sum_squares_Sigma_3 += (lr_results[param]* range_dict[param]**3) ** 2
    
    gamma = invquantile * np.sqrt(sum_squares_Sigma_1 * sum_squares_Sigma_3) / sum_squares_Sigma_2
    
    for param in params:
        pess_direction[param] = lr_results[param]*(range_dict[param])**2 / np.sqrt(sum_squares_Sigma_1)
    
    # calculate pessimistic scenario based on available params
    pess_scenario = dict()
    for param in params:
        pess_scenario[param] = np.mean(PARAM_BOUNDS[param]) + gamma * pess_direction[param]
    
    # add default virtual params if not present
    default_virtual_param_vals = {param:(PARAM_BOUNDS[param][1] + PARAM_BOUNDS[param][0])/2 for param in ADDITIONAL_VIRTUAL_PARAMS}
    for virtual_param, val in default_virtual_param_vals.items():
        if virtual_param not in params:
            pess_scenario[virtual_param] = val

    return gamma, pess_scenario